# Entity Alignment: Hash Approach

This notebook presents a method for entity alignment and merging using sentence embeddings and hashing as an attempt for a more efficient processing. This deals with the problem of merging/entity alignment for our knowledge graph. The notebook includes a hashing function to group similar entity types. Through practical examples, it illustrates the process of merging entities with different similarities, and generates 'also known as' relations for entities with different names but identified as similar. This approach should be further revised.

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 873.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=94a331d8c1900b87b75e6552bc84c695f61d962fcee47c97c1ce8b087abe5680
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.7 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer, util
import hashlib
from collections import defaultdict

# Initialize the model
merge_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

r1 = {'head': 'Université Lyon 2', 'head_type': 'org', 'type': 'subsidiary', 'tail': 'Paris School of Economics', 'tail_type': 'org'}
r2 = {'head': 'Université Lumière Lyon 2', 'head_type': 'org', 'type': 'part of', 'tail': 'GATE', 'tail_type': 'org'}

# r1 = {'head': 'Napoleon', 'head_type': 'Person', 'type': 'Lead', 'tail': 'War of waterloo', 'tail_type': 'Event'}
# r2 = {'head': 'Napoleon Bonaparte', 'head_type': 'Person', 'type': 'Born', 'tail': 'Corsica', 'tail_type': 'Location'}

relations = [r1, r2]

def encode_relation(relation):
    relation_str = ' '.join([relation['head'], relation['type']])
    return merge_model.encode(relation_str)

def similarity_score(enc1, enc2):
    return util.pytorch_cos_sim(enc1, enc2).item()

def should_merge(rel1, rel2, threshold):
    enc1 = encode_relation(rel1)
    enc2 = encode_relation(rel2)
    sim = similarity_score(enc1, enc2)
    print('Similarity between', rel1['head'], 'and', rel2['head'], 'is :', sim)
    return sim >= threshold

def hash_relation(relation):
    return hashlib.md5(relation['head_type'].encode()).hexdigest()

def align_and_merge_relations(relations):
    hashed_groups = defaultdict(list)
    merged_relations = []
    aka_relations = []

    for relation in relations:
        hash_key = hash_relation(relation)
        hashed_groups[hash_key].append(relation)

    for group in hashed_groups.values():
        merged_relations.extend(merge_group(group, aka_relations))

    return merged_relations, aka_relations

def merge_group(group, aka_relations, threshold=0.8):
    merged = []

    while group:
        base = group.pop()
        merge_candidates = [base]

        for other in list(group):
            if should_merge(base, other, threshold):
                merge_candidates.append(other)
                group.remove(other)
                # 'also known as' relation if names are different
                if base['head'] != other['head']:
                    aka_relations.append({'head': base['head'], 'type': 'also known as', 'tail': other['head']})

        merged_head = merge_candidates[0]['head']
        merged_type = merge_candidates[0]['type']
        tails = {cand['tail'] for cand in merge_candidates}
        merged_tail = ', '.join(tails)

        merged.append({'head': merged_head, 'head_type': 'org', 'type': merged_type, 'tail': merged_tail, 'tail_type': 'org'})

    return merged

merged_relations, aka_relations = align_and_merge_relations(relations)
print("Merged Relations:", merged_relations)
print("Also Known As Relations:", aka_relations)


Similarity between Université Lumière Lyon 2 and Université Lyon 2 is : 0.8301360607147217
Merged Relations: [{'head': 'Université Lumière Lyon 2', 'head_type': 'org', 'type': 'part of', 'tail': 'Paris School of Economics, GATE', 'tail_type': 'org'}]
Also Known As Relations: [{'head': 'Université Lumière Lyon 2', 'type': 'also known as', 'tail': 'Université Lyon 2'}]
